# ScyllaDB Store - Advanced Features

This notebook demonstrates all advanced production-ready features including:
- Health checks
- Prometheus metrics export
- Circuit breaker pattern
- Connection warmup
- Metrics API
- Advanced configuration

## Setup

Initialize the store with advanced configuration options.

In [ ]:
import asyncio
from datetime import datetime, timezone
from scylladb_store import AsyncScyllaDBStore, TTLConfig
from cassandra.cluster import Cluster

# Configuration
CONTACT_POINTS = ["127.0.0.1"]
KEYSPACE = "advanced_features_store"

print("✓ Imports successful")

## 1. Basic Store Initialization

Create a store with default configuration.

In [ ]:
# Create cluster and session
cluster = Cluster(CONTACT_POINTS)
session = await asyncio.get_event_loop().run_in_executor(None, cluster.connect)

# Create store with default configuration
store = AsyncScyllaDBStore(
    session=session,
    keyspace=KEYSPACE,
    ttl=TTLConfig(refresh_on_read=True)
)

# Setup database
await store.setup()
print(f"✓ Store initialized with keyspace: {KEYSPACE}")

## 2. Health Check

Monitor the health of your ScyllaDB connection and store.

In [ ]:
# Perform health check
health = await store.health_check()

print(f"Overall Status: {health['status'].upper()}")
print(f"Latency: {health['latency_ms']:.2f}ms\n")

print("Health Checks:")
for check_name, result in health['checks'].items():
    status_icon = "✓" if result['status'] == 'healthy' else "✗"
    print(f"  {status_icon} {check_name}: {result['message']}")

## 3. Connection Warmup

Pre-establish connections for faster first requests.

In [ ]:
# Warmup with 20 concurrent queries
print("Starting connection warmup...")
await store.warmup_connections(num_queries=20)
print("✓ Warmup complete - connections are ready")

## 4. Circuit Breaker Pattern

Prevent cascading failures by enabling the circuit breaker.

In [ ]:
# Enable circuit breaker with custom thresholds
# Note: This is a synchronous method, not async
store.enable_circuit_breaker(
    failure_threshold=5,      # Open circuit after 5 failures
    success_threshold=3,      # Close after 3 successes in half-open state
    timeout_seconds=30.0      # Wait 30s before trying again
)

print("✓ Circuit breaker enabled")
print(f"  Failure threshold: 5")
print(f"  Success threshold: 3")
print(f"  Timeout: 30s")

# Check circuit breaker state (also synchronous)
state = store.circuit_breaker.get_state()
print(f"\nCircuit breaker state: {state['state']}")
print(f"Failure count: {state['failure_count']}")
print(f"Success count: {state['success_count']}")

## 5. Metrics API

Access real-time metrics about your store operations.

In [ ]:
# Create some test data to generate metrics
test_ops = [
    {"namespace": ("metrics", "test", "1"), "key": "data", "value": {"id": 1}},
    {"namespace": ("metrics", "test", "2"), "key": "data", "value": {"id": 2}},
    {"namespace": ("metrics", "test", "3"), "key": "data", "value": {"id": 3}},
]

for op in test_ops:
    await store.aput(**op)

print("✓ Test data created")

In [ ]:
# Get metrics statistics
metrics = await store.get_metrics()

print("=== Query Metrics ===")
print(f"Total queries: {metrics['total_queries']}")
print(f"Average latency: {metrics['avg_latency_ms']:.2f}ms")
print(f"Total errors: {metrics['total_errors']}")
print(f"Error rate: {metrics['error_rate']:.1%}")

print("\n=== Operations by Type ===")
for operation, count in metrics['operations'].items():
    print(f"  {operation}: {count}")

print("\n=== Errors by Type ===")
if metrics['error_types']:
    for error_type, count in metrics['error_types'].items():
        print(f"  {error_type}: {count}")
else:
    print("  No errors recorded")

## 6. Prometheus Metrics Export

Export metrics in Prometheus format for monitoring systems.

In [ ]:
# Export metrics in Prometheus format
prometheus_metrics = await store.export_prometheus_metrics()

print("=== Prometheus Metrics ===")
print(prometheus_metrics)
print("\n✓ These metrics can be scraped by Prometheus")

## 7. Reset Metrics

Clear accumulated metrics for a fresh start.

In [ ]:
# Get metrics before reset
before = await store.get_metrics()
print(f"Queries before reset: {before['total_queries']}")

# Reset metrics
await store.reset_metrics()
print("✓ Metrics reset")

# Get metrics after reset
after = await store.get_metrics()
print(f"Queries after reset: {after['total_queries']}")

## 8. Advanced Configuration

Create a store with custom connection pool and performance settings.

In [ ]:
# Create a new store with advanced configuration
# Note: Connection pool settings are configured at the Cluster level
from cassandra.cluster import ExecutionProfile
from cassandra.policies import DCAwareRoundRobinPolicy

# Create execution profile with custom settings
profile = ExecutionProfile(
    load_balancing_policy=DCAwareRoundRobinPolicy(),
    request_timeout=20.0  # 20s request timeout
)

# Create cluster with advanced configuration
advanced_cluster = Cluster(
    CONTACT_POINTS,
    protocol_version=4,
    executor_threads=8,  # More threads for async callbacks
    compression=True,  # Enable lz4 compression
    connect_timeout=10.0,  # 10s connect timeout
    execution_profiles={'advanced': profile}
)

# Connect and create session
advanced_session = await asyncio.get_event_loop().run_in_executor(
    None, advanced_cluster.connect
)

# Create store with TTL configuration
advanced_store = AsyncScyllaDBStore(
    session=advanced_session,
    keyspace="advanced_config_store",
    ttl=TTLConfig(
        default_ttl=3600.0,      # 1 hour default TTL
        refresh_on_read=True     # Refresh TTL on read
    )
)

await advanced_store.setup()
print("✓ Advanced store created with custom configuration")
print("  - Executor threads: 8")
print("  - Compression: lz4 enabled")
print("  - Connect timeout: 10s")
print("  - Request timeout: 20s")
print("  - Default TTL: 3600s (1 hour)")

## 9. Error Handling

Demonstrate proper error handling with custom exceptions.

In [ ]:
from scylladb_store import (
    StoreConnectionError,
    StoreQueryError,
    StoreValidationError,
    StoreTimeoutError
)

# Example 1: Validation error
try:
    # Try to create a namespace that's too deep
    deep_namespace = tuple([f"level{i}" for i in range(20)])  # 20 levels deep
    await store.aput(
        namespace=deep_namespace,
        key="test",
        value={"data": "test"}
    )
except StoreValidationError as e:
    print(f"✓ Caught validation error: {e}")
    print(f"  Field: {e.field}")

# Example 2: Validation error for large value
try:
    # Try to store a value that's too large (>10MB)
    large_value = {"data": "x" * (11 * 1024 * 1024)}  # 11MB
    await store.aput(
        namespace=("test",),
        key="large",
        value=large_value
    )
except StoreValidationError as e:
    print(f"\n✓ Caught validation error: {e}")
    print(f"  Field: {e.field}")

## 10. Monitoring Slow Queries

The store automatically detects and logs slow queries (>100ms).

In [ ]:
# Create a large dataset to potentially trigger slow queries
print("Creating large dataset...")
for i in range(100):
    await store.aput(
        namespace=("performance", "test"),
        key=f"item_{i}",
        value={"id": i, "data": f"test data {i}"}
    )

print("✓ Dataset created")

# Perform a search that might be slow
print("\nPerforming search...")
results = await store.asearch(
    ("performance",),  # positional-only argument
    limit=100
)
print(f"✓ Found {len(results)} items")

# Check metrics for slow queries
metrics = await store.get_metrics()
print(f"\nAverage latency: {metrics['avg_latency_ms']:.2f}ms")
print("(Check logs for any slow query warnings >100ms)")

## 11. Circuit Breaker in Action

Demonstrate circuit breaker behavior during failures.

In [ ]:
# Circuit breaker state tracking
print("Circuit Breaker Demo\n")

# Initial state
state = store.circuit_breaker.get_state()
print(f"Initial state: {state['state']}")
print(f"  Failure count: {state['failure_count']}")

# Perform some successful operations
for i in range(5):
    await store.aput(
        namespace=("circuit", "test"),
        key=f"item_{i}",
        value={"id": i}
    )

# Check state after successful operations
state = store.circuit_breaker.get_state()
print(f"\nAfter successful operations: {state['state']}")
print(f"  Failure count: {state['failure_count']}")

# Note: Circuit breaker will open only after consecutive failures
# In production, this prevents cascading failures when database is down
print("\n✓ Circuit breaker is monitoring all operations")

## 12. TTL (Time To Live) Management

Demonstrate automatic expiration of data.

In [ ]:
# Create an item with 10-second TTL
print("Creating item with 10-second TTL...")
await store.aput(
    namespace=("ttl", "test"),
    key="temporary",
    value={"message": "This will expire in 10 seconds"},
    ttl=10.0  # TTL in seconds
)

# Verify it exists
item = await store.aget(("ttl", "test"), "temporary")
print(f"✓ Item created: {item.value['message']}")

# Wait for expiration
print("\nWaiting 12 seconds for expiration...")
await asyncio.sleep(12)

# Try to retrieve expired item
expired = await store.aget(("ttl", "test"), "temporary")
print(f"Item after expiration: {expired}")
print("✓ Item expired as expected" if expired is None else "✗ Item still exists")

## 13. Execution Profiles

Configure different execution settings for different query types.

In [ ]:
# Create standard execution profiles for common use cases
print("Creating standard execution profiles...")
store.create_standard_profiles()

print("\n✓ Created 4 standard profiles:")
print("  1. strong_reads    - QUORUM consistency, 30s timeout (critical reads)")
print("  2. fast_writes     - ONE consistency, 5s timeout (high throughput)")
print("  3. lwt_operations  - SERIAL consistency, 30s timeout (atomic ops)")
print("  4. analytics       - ALL consistency, 60s timeout (analytics)")

# Create a custom profile
from cassandra.query import ConsistencyLevel

store.add_execution_profile(
    'custom_profile',
    consistency_level=ConsistencyLevel.LOCAL_QUORUM,
    request_timeout=15.0
)
print("\n✓ Created custom profile: custom_profile (LOCAL_QUORUM, 15s)")

print("\n✓ Execution profiles allow fine-tuned control over query execution")

## 14. Lightweight Transactions (LWT)

Demonstrate atomic conditional operations for preventing race conditions.

In [ ]:
# Example 1: Conditional Insert (IF NOT EXISTS)
# Useful for preventing duplicate inserts and implementing distributed locks

print("=== Conditional Insert (IF NOT EXISTS) ===\n")

# First attempt - should succeed
success = await store.aput_if_not_exists(
    namespace=("locks",),
    key="resource_123",
    value={"owner": "worker_1", "acquired_at": "2025-10-03T00:00:00"},
    ttl=60.0
)
print(f"First insert attempt: {'SUCCESS' if success else 'FAILED'}")

# Second attempt - should fail (key already exists)
success = await store.aput_if_not_exists(
    namespace=("locks",),
    key="resource_123",
    value={"owner": "worker_2", "acquired_at": "2025-10-03T00:00:10"}
)
print(f"Second insert attempt: {'SUCCESS' if success else 'FAILED (key exists)'}")

# Verify current value
item = await store.aget(("locks",), "resource_123")
print(f"\nCurrent lock owner: {item.value['owner']}")
print("✓ Lock correctly held by first worker")

In [ ]:
# Example 2: Conditional Update (IF EXISTS)
# Ensures you only update existing keys, preventing accidental creation

print("\n=== Conditional Update (IF EXISTS) ===\n")

# Try to update non-existent key - should fail
success = await store.aupdate_if_exists(
    namespace=("users",),
    key="999",
    value={"name": "Ghost User", "status": "inactive"}
)
print(f"Update non-existent user: {'SUCCESS' if success else 'FAILED (does not exist)'}")

# Create a user first
await store.aput(
    namespace=("users",),
    key="123",
    value={"name": "Alice", "status": "active"}
)
print("Created user 123")

# Now update - should succeed
success = await store.aupdate_if_exists(
    namespace=("users",),
    key="123",
    value={"name": "Alice", "status": "inactive"}
)
print(f"Update existing user: {'SUCCESS' if success else 'FAILED'}")

# Verify update
item = await store.aget(("users",), "123")
print(f"\nUser 123 status: {item.value['status']}")
print("✓ Conditional update works correctly")

In [ ]:
# Example 3: Compare-And-Set (CAS)
# Atomic read-modify-write pattern for optimistic locking

print("\n=== Compare-And-Set (CAS) ===\n")

# Initialize a counter
await store.aput(
    namespace=("counters",),
    key="page_views",
    value={"count": 0}
)
print("Initialized counter at 0")

# Simulate atomic increment
for i in range(5):
    # Read current value
    item = await store.aget(("counters",), "page_views")
    current = item.value
    
    # Compute new value
    new = {"count": current["count"] + 1}
    
    # Atomic update only if value hasn't changed
    success = await store.acompare_and_set(
        namespace=("counters",),
        key="page_views",
        expected_value=current,
        new_value=new
    )
    
    if success:
        print(f"Increment {i+1}: {current['count']} -> {new['count']}")
    else:
        print(f"Increment {i+1}: FAILED (concurrent modification)")

# Verify final count
item = await store.aget(("counters",), "page_views")
print(f"\nFinal count: {item.value['count']}")
print("✓ CAS ensures atomicity even with concurrent updates")

In [ ]:
# Example 4: Conditional Delete
# Delete only if conditions are met

print("\n=== Conditional Delete ===\n")

# Create test items
await store.aput(("temp",), "item1", {"status": "active"})
await store.aput(("temp",), "item2", {"status": "active"})
print("Created 2 test items")

# Try to delete non-existent item - should fail
success = await store.adelete_if_exists(("temp",), "item999")
print(f"\nDelete non-existent item: {'SUCCESS' if success else 'FAILED (does not exist)'}")

# Delete existing item - should succeed
success = await store.adelete_if_exists(("temp",), "item1")
print(f"Delete existing item1: {'SUCCESS' if success else 'FAILED'}")

# Delete with value check - wrong value should fail
success = await store.adelete_if_value(
    namespace=("temp",),
    key="item2",
    expected_value={"status": "inactive"}
)
print(f"Delete item2 with wrong value: {'SUCCESS' if success else 'FAILED (value mismatch)'}")

# Delete with correct value - should succeed
success = await store.adelete_if_value(
    namespace=("temp",),
    key="item2",
    expected_value={"status": "active"}
)
print(f"Delete item2 with correct value: {'SUCCESS' if success else 'FAILED'}")

# Verify deletions
item1 = await store.aget(("temp",), "item1")
item2 = await store.aget(("temp",), "item2")
print(f"\nitem1 exists: {item1 is not None}")
print(f"item2 exists: {item2 is not None}")
print("✓ Conditional deletes work correctly")

In [ ]:
# Example 5: Distributed Lock Pattern
# Real-world use case: coordinating work across multiple workers

print("\n=== Distributed Lock Pattern ===\n")

async def acquire_lock(resource_id: str, worker_id: str, ttl: float = 30.0) -> bool:
    """Try to acquire a distributed lock."""
    return await store.aput_if_not_exists(
        namespace=("locks",),
        key=resource_id,
        value={"owner": worker_id, "acquired_at": datetime.now(timezone.utc).isoformat()},
        ttl=ttl
    )

async def release_lock(resource_id: str, worker_id: str) -> bool:
    """Release a lock only if we own it."""
    item = await store.aget(("locks",), resource_id)
    if item and item.value.get("owner") == worker_id:
        return await store.adelete_if_value(
            namespace=("locks",),
            key=resource_id,
            expected_value=item.value
        )
    return False

# Simulate multiple workers trying to acquire lock
print("Worker 1 acquiring lock...")
lock1 = await acquire_lock("task_processor", "worker_1")
print(f"Worker 1: {'✓ Lock acquired' if lock1 else '✗ Failed'}")

print("\nWorker 2 trying to acquire same lock...")
lock2 = await acquire_lock("task_processor", "worker_2")
print(f"Worker 2: {'✓ Lock acquired' if lock2 else '✗ Failed (already locked)'}")

# Worker 2 cannot release Worker 1's lock
print("\nWorker 2 trying to release Worker 1's lock...")
released = await release_lock("task_processor", "worker_2")
print(f"Worker 2 release: {'✓ Success' if released else '✗ Failed (not owner)'}")

# Worker 1 can release its own lock
print("\nWorker 1 releasing its lock...")
released = await release_lock("task_processor", "worker_1")
print(f"Worker 1 release: {'✓ Success' if released else '✗ Failed'}")

# Now Worker 2 can acquire
print("\nWorker 2 trying again...")
lock2 = await acquire_lock("task_processor", "worker_2")
print(f"Worker 2: {'✓ Lock acquired' if lock2 else '✗ Failed'}")

print("\n✓ Distributed locking prevents race conditions")

## 15. ScyllaDB Shard Awareness

Check ScyllaDB-specific shard-aware optimizations.

In [ ]:
# Get ScyllaDB shard awareness information
shard_info = store.get_shard_awareness_info()

print("=== ScyllaDB Shard Awareness ===\n")

if shard_info['is_shard_aware']:
    print("✓ Shard awareness: ENABLED")
    print("  This optimizes performance by routing queries directly to the correct shard")
    print("  Reduces latency by eliminating inter-shard communication")
    
    if shard_info['shard_stats']:
        print(f"\n  Shard statistics:")
        print(f"  {shard_info['shard_stats']}")
else:
    print("✗ Shard awareness: NOT AVAILABLE")
    print("  (This is a ScyllaDB-specific optimization)")

print(f"\n=== Cluster Metadata ===")
print(f"Contact points: {shard_info['cluster_metadata']['contact_points']}")
print(f"Protocol version: {shard_info['cluster_metadata']['protocol_version']}")
print(f"Compression: {shard_info['cluster_metadata']['compression']}")

print("\n✓ ScyllaDB-specific optimizations configured")

## 16. Query Paging for Large Result Sets

Efficiently handle large result sets with automatic paging.

In [ ]:
# Create a dataset to demonstrate paging
print("Creating dataset with 500 items...")
for i in range(500):
    await store.aput(
        namespace=("paging", "test"),
        key=f"item_{i:04d}",
        value={"id": i, "data": f"test data {i}"}
    )   
print("✓ Dataset created\n")
# Search with default paging (fetch_size=5000)
print("=== Search with Default Paging (fetch_size=5000) ===")
results = await store.asearch(("paging",), limit=500)
print(f"Retrieved {len(results)} items")
print("✓ All items fit in single page (500 < 5000)\n")
# Search with small page size (fetch_size=50)
print("=== Search with Small Page Size (fetch_size=50) ===")
results = await store.asearch(
    ("paging",),
    limit=500,
    fetch_size=50  # Fetch 50 rows per page
)
print(f"Retrieved {len(results)} items")
print("✓ Driver automatically fetched 10 pages (50 rows each)")
print("✓ Reduces memory usage - good for memory-constrained environments\n")
# Search with medium page size
print("=== Search with Medium Page Size (fetch_size=200) ===")
results = await store.asearch(
    ("paging",),
    limit=500,
    fetch_size=200
)
print(f"Retrieved {len(results)} items")
print("✓ Driver fetched 3 pages (200 rows each)")
print("✓ Balanced approach - fewer round trips than fetch_size=50\n")
print("=== Paging Trade-offs ===")
print("  Small fetch_size (e.g., 50):   Less memory, more round trips")
print("  Medium fetch_size (e.g., 200): Balanced memory and round trips")
print("  Large fetch_size (e.g., 5000): More memory, fewer round trips")
print("  Default (5000):                Good balance for most use cases")
print("\n✓ Choose based on: dataset size, memory constraints, latency requirements")


## 17. Atomic Batch Operations

Demonstrate true atomic batch processing with LOGGED and UNLOGGED batch types.

In [ ]:
# Test 1: UNLOGGED Batch (Default - Best Performance)
print("=== UNLOGGED Batch (Default) ===\n")

from scylladb_store import PutOp

# Create batch of PUT operations
ops = [
    PutOp(namespace=('batch', 'unlogged'), key=f'item_{i}', value={'id': i, 'data': f'test {i}'})
    for i in range(10)
]

# Execute with default UNLOGGED batch
results = await store.abatch(ops)
print(f"✓ Executed {len(ops)} operations atomically")
print(f"  Batch type: UNLOGGED (best performance)")
print(f"  Atomicity: Within partition")

# Verify data
item = await store.aget(('batch', 'unlogged'), 'item_5')
print(f"  Verified item_5: {item.value}")

print("\n" + "="*60)

In [ ]:
# Test 2: LOGGED Batch (Full Atomicity)
print("\n=== LOGGED Batch (Full Atomicity) ===\n")

# Create batch for critical operations requiring full atomicity
ops_logged = [
    PutOp(namespace=('batch', 'logged'), key=f'critical_{i}', value={'id': i, 'type': 'critical'})
    for i in range(5)
]

# Execute with LOGGED batch for full atomicity
results = await store.abatch(ops_logged, batch_type='LOGGED')
print(f"✓ Executed {len(ops_logged)} operations atomically")
print(f"  Batch type: LOGGED (full atomicity across partitions)")
print(f"  Use case: Financial transactions, critical updates")

# Verify data
item = await store.aget(('batch', 'logged'), 'critical_2')
print(f"  Verified critical_2: {item.value}")

print("\n" + "="*60)

In [ ]:
# Test 4: Batch with Retry Logic
print("\n=== Batch with Retry Logic ===\n")

# Create batch with retry configuration
ops_retry = [
    PutOp(namespace=('batch', 'retry'), key=f'item_{i}', value={'id': i, 'resilient': True})
    for i in range(10)
]

# Execute with retry configuration for resilience
results = await store.abatch(
    ops_retry,
    max_retries=3,          # Retry up to 3 times
    retry_delay=0.1,        # Start with 100ms delay
    retry_backoff=2.0       # Double delay after each retry
)

print(f"✓ Executed {len(results)} operations with retry protection")
print(f"  Retry configuration:")
print(f"    - Max retries: 3")
print(f"    - Initial delay: 0.1s")
print(f"    - Backoff: 2.0x (exponential)")
print(f"  Delays: 0.1s → 0.2s → 0.4s")
print(f"  Retryable errors: Timeout, Unavailable replicas")

print("\n✓ Batch operations are resilient to transient failures")

In [ ]:
# Test 3: Mixed Operations (Falls back to Concurrent Execution)
print("\n=== Mixed Operations Batch ===\n")

from scylladb_store import GetOp

# Create batch with mixed operation types
mixed_ops = [
    PutOp(namespace=('batch', 'mixed'), key='new_item', value={'type': 'put'}),
    GetOp(namespace=('batch', 'unlogged'), key='item_5'),
    PutOp(namespace=('batch', 'mixed'), key='another', value={'type': 'put2'}),
]

# Execute - automatically uses concurrent execution for mixed operations
results = await store.abatch(mixed_ops)
print(f"✓ Executed {len(mixed_ops)} operations")
print(f"  Mode: Concurrent (mixed operation types)")
print(f"  Note: batch_type parameter ignored for mixed operations")
print(f"  GET result: {results[1].value if results[1] else None}")

print("\n✓ abatch() intelligently selects atomic vs concurrent execution")

In [ ]:
# Test 4: Batch with TTL
print("\n=== Batch with TTL ===\n")

# Create batch with TTL for temporary data
ops_ttl = [
    PutOp(namespace=('batch', 'ttl'), key=f'temp_{i}', value={'id': i}, ttl=60.0)
    for i in range(5)
]

# Execute batch with TTL
results = await store.abatch(ops_ttl)
print(f"✓ Executed {len(ops_ttl)} operations with 60s TTL")
print(f"  All items will expire in 60 seconds")

# Verify data
item = await store.aget(('batch', 'ttl'), 'temp_0')
print(f"  Verified temp_0: {item.value}")

print("\n" + "="*60)

## 18. Comprehensive Health Check

Perform a final health check to verify all systems.

In [ ]:
# Final comprehensive health check
health = await store.health_check()

print("=== Final Health Check ===")
print(f"Overall Status: {health['status'].upper()}")
print(f"Check Latency: {health['latency_ms']:.2f}ms\n")

for check_name, result in health['checks'].items():
    status_icon = "✓" if result['status'] == 'healthy' else "✗"
    print(f"{status_icon} {check_name.upper()}")
    print(f"  {result['message']}")
    if 'details' in result:
        for key, value in result['details'].items():
            print(f"  - {key}: {value}")
    print()

## 19. Final Metrics Summary

Review all metrics collected during this session.

In [ ]:
# Get final metrics
metrics = await store.get_metrics()

print("=== Final Metrics Summary ===")
print(f"\nTotal Operations: {metrics['total_queries']}")
print(f"Average Latency: {metrics['avg_latency_ms']:.2f}ms")
print(f"Error Rate: {metrics['error_rate']:.2%}")

print("\nOperations Breakdown:")
for op_type, count in sorted(metrics['operations'].items(), key=lambda x: x[1], reverse=True):
    print(f"  {op_type}: {count}")

if metrics['error_types']:
    print("\nErrors Encountered:")
    for error_type, count in metrics['error_types'].items():
        print(f"  {error_type}: {count}")
else:
    print("\n✓ No errors encountered")

print(f"\n✓ All advanced features demonstrated successfully")

## 20. Cleanup (Optional)

Clean up test data.

In [ ]:
# Uncomment to clean up test data
async def cleanup():
    namespaces = await store.alist_namespaces()
    
    for ns in namespaces:
        items = await store.asearch(ns, limit=1000)
        for item in items:
            await store.adelete(item.namespace, item.key)
    
    print("✓ All test data cleaned up")

await cleanup()

## Summary

This notebook demonstrated:

1. ✓ **Health Checks** - Monitor store and database health
2. ✓ **Connection Warmup** - Pre-establish connections for better performance
3. ✓ **Circuit Breaker** - Prevent cascading failures
4. ✓ **Metrics API** - Real-time operation statistics
5. ✓ **Prometheus Export** - Integration with monitoring systems
6. ✓ **Advanced Configuration** - Custom connection pools and settings
7. ✓ **Error Handling** - Comprehensive exception hierarchy
8. ✓ **Slow Query Detection** - Automatic performance monitoring
9. ✓ **TTL Management** - Automatic data expiration
10. ✓ **Production-Ready** - All best practices implemented

### Key Features for Production:

- **Native Async I/O**: True asyncio integration with AsyncioConnection
- **Observability**: Comprehensive metrics and health checks
- **Resilience**: Circuit breaker pattern prevents cascading failures
- **Performance**: Connection warmup, compression, optimized connection pools
- **Monitoring**: Prometheus metrics export for production monitoring
- **Error Handling**: Rich exception hierarchy with context
- **Validation**: Input validation prevents data corruption
- **Flexibility**: Highly configurable for different workloads